In [1]:
from cnc import CNC, simulate_from_distribution
from utils import Pauli, qutip_simuation
from vertex_decomposition import find_vertex_decomposition, find_cnc_vertex_decomposition

#### Simulation Example

In [2]:
num_simulations = 2048
cnc_set = { Pauli("II"),
    Pauli("IZ"),
    Pauli("ZI"),
    Pauli("ZZ"),
    Pauli("YI"),
    Pauli("YZ"),
    Pauli("XI"),
    Pauli("XZ"),
}

value_assignment1 = {pauli: 0 for pauli in cnc_set}

cnc_1  = CNC(value_assignment1)

value_assignment2 = {
    Pauli("II"): 0,
    Pauli("IZ"): 0,
    Pauli("ZI"): 0,
    Pauli("ZZ"): 0,
    Pauli("YI"): 1,
    Pauli("YZ"): 1,
    Pauli("XI"): 1,
    Pauli("XZ"): 1,
}

cnc3 = CNC({
    Pauli("II"): 0,
    Pauli("IZ"): 0,
    Pauli("ZI"): 0,
    Pauli("ZZ"): 0,
})

cnc_2 = CNC(value_assignment2)

measurements = [
    Pauli("YY"),
    Pauli("ZI"),
]

distribution = {cnc3: 1}

counts_cnc = simulate_from_distribution(
    distribution, measurements, num_simulations
)

In [3]:
counts_cnc

{'00': 542, '01': 512, '10': 489, '11': 505}

In [4]:
from qutip import Qobj, identity, sigmaz, sigmay, tensor

II = tensor(identity(2), identity(2))
ZI = tensor(sigmaz(), identity(2))
IZ = tensor(identity(2), sigmaz())
ZZ = tensor(sigmaz(), sigmaz())
YY = tensor(sigmay(), sigmay())

rho = 1 / 4 * (II + ZI + IZ + ZZ)

measurements = [YY, ZI]

num_simulations = 2048
counts_qutip = qutip_simuation(rho, measurements, num_simulations)

In [5]:
counts_qutip

{'00': 479, '01': 516, '10': 533, '11': 520}

#### Finding initial distribution

In [6]:
from utils import load_all_maximal_cncs_matrix

all_cncs_2 = load_all_maximal_cncs_matrix(2)
all_cncs_3 = load_all_maximal_cncs_matrix(3)

In [7]:
all_cncs_2.shape

(16, 492)

In [8]:
all_cncs_2

array([[ 1,  1,  1, ...,  1,  1,  1],
       [ 0, -1,  0, ...,  1,  0,  0],
       [ 0,  0,  0, ...,  0,  0, -1],
       ...,
       [-1,  0,  0, ...,  0, -1, -1],
       [-1,  1,  0, ...,  0,  0,  0],
       [ 0, -1,  0, ...,  0,  0, -1]])

In [9]:
all_cncs_3.shape

(64, 72216)

Check whether pauli basis representation functions work as intended.

In [10]:
print(all_cncs_2[:, 0])
cnc = CNC.from_pauli_basis_representation(all_cncs_2[:, 0])
print(cnc.gamma)
cnc.get_pauli_basis_representation()

[ 1  0  0 -1  0 -1  1  0 -1  0  0  1  0 -1 -1  0]
{Pauli Operator: II: 0, Pauli Operator: IZ: 1, Pauli Operator: XX: 1, Pauli Operator: XY: 0, Pauli Operator: YI: 1, Pauli Operator: YZ: 0, Pauli Operator: ZX: 1, Pauli Operator: ZY: 1}


array([ 1,  0,  0, -1,  0, -1,  1,  0, -1,  0,  0,  1,  0, -1, -1,  0])

In [11]:
import numpy as np
initial_state = np.zeros(16)
initial_state[0] = 1

Find initial distribution

In [12]:
is_convex, distribution = find_vertex_decomposition(initial_state, all_cncs_2)

In [13]:
is_convex

True

In [14]:
for decomp_element in distribution:
    print(decomp_element)

DecompositionElement(operator=array([1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0]), probability=0.0633803)
DecompositionElement(operator=array([ 1,  0,  0,  0,  0,  1,  0,  0,  0,  0, -1,  0,  0,  0,  0,  1]), probability=0.084507)
DecompositionElement(operator=array([ 1,  0,  0,  0,  0,  0,  0,  1,  0,  0, -1,  0,  0, -1,  0,  0]), probability=0.0704225)
DecompositionElement(operator=array([ 1,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0, -1,  0]), probability=0.0915493)
DecompositionElement(operator=array([ 1,  0, -1,  0,  0,  0,  0,  0,  1,  0, -1,  0,  0,  0,  0,  0]), probability=0.00704225)
DecompositionElement(operator=array([ 1,  0,  0, -1,  1,  0,  0, -1,  0,  0,  0,  0,  0,  0,  0,  0]), probability=0.161972)
DecompositionElement(operator=array([1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0]), probability=0.0774648)
DecompositionElement(operator=array([ 1,  0,  0,  0,  0,  0,  0,  1,  0, -1,  0,  0,  0,  0,  1,  0]), probability=0.0492958)
DecompositionElement(oper

In [15]:
result_vec = np.zeros(16)
for i, decomp_element in enumerate(distribution):
    op = decomp_element.operator
    prob = decomp_element.probability
    result_vec += prob * op

result_vec

array([ 9.9999975e-01, -3.0000000e-07, -5.0000000e-08, -2.0000000e-07,
        5.0000000e-07,  3.0000000e-07,  0.0000000e+00, -2.0000000e-07,
       -5.0000000e-08,  0.0000000e+00,  5.0000000e-08,  0.0000000e+00,
        1.0000000e-07,  0.0000000e+00,  1.0000000e-07,  0.0000000e+00])

In [16]:
is_convex, distribution = find_cnc_vertex_decomposition(initial_state)

In [17]:
for i, decomp_element in enumerate(distribution):
    cnc = decomp_element.operator
    prob = decomp_element.probability
    print(f"CNC {i}: with probability {prob}")
    for pauli, value in cnc.gamma.items():
        print(f"{pauli}: {value}")

CNC 0: with probability 0.0633803
Pauli Operator: II: 0
Pauli Operator: XX: 0
Pauli Operator: YZ: 0
Pauli Operator: ZY: 0
CNC 1: with probability 0.084507
Pauli Operator: II: 0
Pauli Operator: XX: 0
Pauli Operator: YY: 1
Pauli Operator: ZZ: 0
CNC 2: with probability 0.0704225
Pauli Operator: II: 0
Pauli Operator: XZ: 0
Pauli Operator: YY: 1
Pauli Operator: ZX: 1
CNC 3: with probability 0.0915493
Pauli Operator: II: 0
Pauli Operator: IY: 1
Pauli Operator: ZI: 0
Pauli Operator: ZY: 1
CNC 4: with probability 0.00704225
Pauli Operator: II: 0
Pauli Operator: IY: 1
Pauli Operator: YI: 0
Pauli Operator: YY: 1
CNC 5: with probability 0.161972
Pauli Operator: II: 0
Pauli Operator: IZ: 1
Pauli Operator: XI: 0
Pauli Operator: XZ: 1
CNC 6: with probability 0.0774648
Pauli Operator: II: 0
Pauli Operator: IY: 0
Pauli Operator: ZI: 0
Pauli Operator: ZY: 0
CNC 7: with probability 0.0492958
Pauli Operator: II: 0
Pauli Operator: XZ: 0
Pauli Operator: YX: 1
Pauli Operator: ZY: 0
CNC 8: with probability 0